In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from pyquery import PyQuery as pq
from pymongo import MongoClient


In [63]:
class JdSpider(object):
    def __init__(self, key_word, database='test', collection='test'):
        """初始化变量"""
        # 获取想要搜索的内容
        self.key_word = key_word
        client = MongoClient(host='localhost', port=27017)
        self.collection = client[database][collection]
        self.browser = webdriver.Chrome()
        self.wait = WebDriverWait(self.browser, 10)

    def init_browser(self):
        self.browser.get('https://www.jd.com/')
        # 获取输入框，并输入搜索内容
        search_input = self.wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#key')))
        search_input.clear()
        # 回车，搜索
        search_input.send_keys(self.key_word)
        search_input.send_keys(Keys.ENTER)
        time.sleep(2)
        # 翻到网页最下面
        self.browser.execute_script(
            'window.scrollTo(0, document.body.scrollHeight)')
        # 执行js，获取最大页码数
        time.sleep(2)
        max_page = self.wait.until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, '#J_bottomPage .p-skip em > b')))
        try:
            max_page = int(max_page.text)
            print('初始化浏览器，并获取最大页码数成功：max_page:{}'.format(max_page))
        except:
            print('max_page 获取失败!!')
            max_page = 1
        return max_page

    def get_page_source(self):
        """获取网页源代码"""
        try:
            # 让滚动条滚到最下面获取所有的li
            self.browser.execute_script(
                'window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(2)
            self.wait.until(
                EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, '#J_goodsList ul')))
            print('获取网页源代码成功')
            page_source = self.browser.page_source
            return page_source
        except TimeoutException as e:
            print('Time out')
            get_page_source()
#         except Exception as e:
#             print("Error is {}".format(e))

    def save_to_database(self, product_list):
        for product_data in product_list:
            href_dict = {'href': product_data.get('href')}
            if self.collection.find_one(href_dict):
                self.collection.update_one(href_dict, {'$set': product_data})
            else:
                try:
                    self.collection.insert_one(product_data)
                except Exception as e:
                    print('保存失败')
                    print('error is {}'.format(e))
        print('保存成功')

    def parse_url(self, page_source):
        """解析网页，返回相关商品信息"""
        print('开始解析')
        doc = pq(page_source)
        items = doc('#J_goodsList ul li').items()
        #         product_list = []
        items = doc('#J_goodsList .gl-warp .gl-item').items()
        for item in items:
            product_data = {}
            price = item('.p-price strong i').text()
            product_data['href'] = "https:" + item('.p-name a').attr('href')
            product_data['title'] = item('.p-name a').attr('title')  # 获取商品描述
            product_data['price'] = float(price)
            product_data['comment_count'] = item('.p-commit strong ').text()
            product_data['shop'] = item('.p-shop .J_im_icon a').text()
            #             product_list.append(product_data)
            yield product_data
        print('解析成功')

    def turn_page(self, page):
        page_input = self.browser.find_element_by_css_selector(
            '#J_bottomPage .input-txt')
        submit_button = self.browser.find_element_by_css_selector(
            '#J_bottomPage .btn')
        page_input.clear()
        page_input.send_keys(page)
        print('跳转到 {}页'.format(page))
        submit_button.click()

    def run(self):
        """实现主程序"""
        # 0.初始化浏览器
        page = 2
        max_page = self.init_browser()
        for page in range(2, max_page + 2):
            # 1.翻到网页最下面，让所有的内容加载完成，并返回网页源码
            print('开始第{}页'.format(page - 1))
            page_source = self.get_page_source()
            # 2.解析网页源码，返回解析后的商品信息
            product_list = self.parse_url(page_source)
            # 3.保存到数据库
            self.save_to_database(product_list)
            print('结束第{}页'.format(page - 1))
            #           4.翻页（如果当前页码是最大页码数则不翻页）
            if page < max_page:
                self.turn_page(page)

In [64]:
if __name__ == "__main__":
    jd_ipad = JdSpider('ipad', 'jd', 'jd_ipad')
    try:
        jd_ipad.run()
    except Exception as e:
        print('错误是 ', e)
    finally:
        jd_ipad.browser.quit()

初始化浏览器，并获取最大页码数成功：max_page:100
开始第1页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第1页
跳转到 2页
开始第2页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第2页
跳转到 3页
开始第3页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第3页
跳转到 4页
开始第4页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第4页
跳转到 5页
开始第5页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第5页
跳转到 6页
开始第6页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第6页
跳转到 7页
开始第7页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第7页
跳转到 8页
开始第8页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第8页
跳转到 9页
开始第9页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第9页
跳转到 10页
开始第10页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第10页
跳转到 11页


KeyboardInterrupt: 